In [1]:
import pandas as pd
import scipy
import implicit
from implicit import evaluation

In [2]:
df = pd.read_pickle('sessions_counts.pkl')
tracks_df = pd.read_pickle('tracks_df.pkl')
artists_df = pd.read_pickle('artists_df.pkl') 

In [3]:
def track_id_to_artist_id_mapping(track_list):
    tracks = tracks_df['track_id']
    artists = tracks_df['artist_id']
    d = dict(zip(tracks, artists))
    
    artist_list = []
    for track in track_list:
        try:
            artist = d[track]
            artist_list.append(artist)
        except KeyError:
            pass
    
    artist_counter = dict(Counter(artist_list))
    
    return artist_counter

In [4]:
def artist_id_to_artist_name(df, id_):
    '''
    Getting artist name via artist id
    '''
    artist_name = df[df['artist_id'] == id_].iloc[0, 1]
    return artist_name

In [5]:
df = df[0:5000]
user_artists_matrix = pd.json_normalize(df['artist_counts'])
user_artists_matrix.set_index(df.index, inplace=True)
user_artists_matrix.fillna(0, inplace=True)

In [6]:
sparse_matrix = scipy.sparse.csr_matrix(user_artists_matrix.values)
sparse_matrix_train = sparse_matrix[0:4000]
sparse_matrix_test = sparse_matrix[4000:5000]

### ALS (only sessions data)

In [7]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=3008)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(sparse_matrix_train)

In [8]:
# eval algorithm on test dataset
evaluation.mean_average_precision_at_k(model, sparse_matrix_train, sparse_matrix_test, num_threads=0)

0.0019891269841269842

### ALS (sessions data with like advantages)

In [4]:
df = pd.read_pickle('sessions_likes.pkl')
tracks_df = pd.read_pickle('tracks_df.pkl')
artists_df = pd.read_pickle('artists_df.pkl') 

In [5]:
df = df[0:5000]
user_artists_matrix = pd.json_normalize(df['artist_counts'])
user_artists_matrix.set_index(df.index, inplace=True)
user_artists_matrix.fillna(0, inplace=True)

sparse_matrix = scipy.sparse.csr_matrix(user_artists_matrix.values)
sparse_matrix_train = sparse_matrix[0:4000]
sparse_matrix_test = sparse_matrix[4000:5000]

In [17]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=3008)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(sparse_matrix_train)

In [18]:
# eval algorithm on test dataset
evaluation.mean_average_precision_at_k(model, sparse_matrix_train, sparse_matrix_test, num_threads=0)

0.004146785714285715

### Represent results

In [8]:
def recommend_artists_for_user(userid, model, sparse_matrix):
    user_items = sparse_matrix.T.tocsr()
    recommendations = model.recommend(userid, user_items)
    print(f'Best recommendations for user {userid}:\n')
    for rec in recommendations[:5]:
        artist = rec[0]
        print(artist_id_to_artist_name(artists_df, id_=artist))

In [9]:
recommend_artists_for_user(6, model, sparse_matrix_train)

Best recommendations for user 6:

176BIZ
2.01 - One Republic ft Sara Bareilles
1 I Feel Better
1997 A Grand Love Story (club33689845)((CD1)
33Miles


In [10]:
def smilar_with_artist(artist, model):
    print(f'Most smilar artists to {(artist_id_to_artist_name(artists_df, id_=artist))}:\n')
    idx_to_artist = dict(enumerate(list(map(int, user_artists_matrix.columns))))
    idx_to_artist = dict((v,k) for k,v in idx_to_artist.items())
    artist = idx_to_artist[artist]  # getting artists by index in sparse matrix
    
    related = model.similar_items(artist)
    for rel in related[1:5]:
        artist = rel[0]
        print(artist_id_to_artist_name(artists_df, id_=artist))

In [11]:
smilar_with_artist(421930, model)

Most smilar artists to Dragomir Milenkovic:

118_Aqua And Arctic
3Carbine
22-Pistepirkko
37 ESC 2010 - Turkey - Manga - We could be the same


In [12]:
smilar_with_artist(295530, model)

Most smilar artists to Rihana:

11 LIRA DE ORFEO
1RAVOHN
13. Benjamin Shock
2razyPe
